# 惯例的helloworld

首先我们应该对Cpp程序有个大概的概念.

cpp程序需要编译.其源文件(source files)一般以`.cpp`,`.cxx`或者`.cc`为后缀.源文件的作用是定义程序的实现,也就是人能看得懂的写给机器真正执行的文件.

一般微软规范中使用`.cpp`而`linux`中常用`.cc`个人认为`.cpp`更合理.不过其实随便.cpp的编译需要编译器,最常见的就是`GNU g++`命令编译.




这两种形式的输出都为int类型.如果返回0,则代表程序正常退出.返回其它数字的含义则由系统决定.通常,返回非零代表程序异常退出.

下面是名为`helloworld.cpp`的源文件.这个源文件有如下几个部分:

+ 引入头文件,c和c++中有头文件这种设定,一般由`.hpp`或者`.h`为后缀.头文件是编译器处理的文件而不是程序执行的文件,它一般用于:

    + 声明源文件的接口
    + 申明模板
    + 申明一些常量,和自定义类型
    
    头文件的引入就是在引入位置直接复制头文件中的内容,因此也有用来定义要引入的文本的情况.这种情况其实并不是头文件,因此我们在这种情况下使用`.inc`为后缀
   
+ 入口函数main函数.

    所有cpp程序都要有一个入口函数--main函数,它可以是两种形式

    + `int main(int argc, char *argv[])`其中输入:

        + argc: 参数的个数,不给main()函数传递参数时默认值为1，即至少有一个参数为该可执行文件的文件名(含目录).
        + argv: 为指针数组,分别指向各个字符串参数的首地址,其中argv[0]存储的是可执行文件的文件名的首地址.

    + `int main()`


In [12]:
%%writefile codes/src/helloworld/section1/helloworld.cpp
#include <stdio.h>

int main(){
    printf("hello world");
    return 0;
}

Overwriting codes/src/helloworld/section1/helloworld.cpp


In [13]:
!g++ -o codes/bin/helloworld/section1/helloworld codes/src/helloworld/section1/helloworld.cpp

In [14]:
!./codes/bin/helloworld/section1/helloworld

hello world

## 一个更加典型的helloworld程序

Cpp是为

In [23]:
%%writefile codes/src/helloworld/section2/helloworld.cpp
#include <string>
using std::string;
string hello_name(string name){
    return "hello world "+name;
}

Overwriting codes/src/helloworld/section2/helloworld.cpp


In [27]:
%%writefile codes/include/helloworld/section2/helloworld.hpp
#include <string>
using std::string;
string hello_name(string name);

Overwriting codes/include/helloworld/section2/helloworld.hpp


In [30]:
%%writefile codes/src/helloworld/section2/main.cpp
#include <iostream>
#include "helloworld.hpp"
using std::cout;
using std::endl;
int main(){
    cout << hello_name("Cpp!") <<endl;
    return 0;
}

Overwriting codes/src/helloworld/section2/main.cpp


In [31]:
!g++ -I codes/include/helloworld/section2 -o codes/bin/helloworld/section2/helloworld codes/src/helloworld/section2/*.cpp

In [32]:
!codes/bin/helloworld/section2/helloworld

hello world Cpp!
